In [1]:
#@title Allow your drive for local storage and set prefs (2 minutes, run once per session)
import os, sys
os.environ["LANGCHAIN_HANDLER"] = "langchain"

#@markdown OpenAI model name (recommend text-davinci-003)
OPENAI_MODEL_NAME = "text-davinci-003" #@param {type:"string"}

#@markdown Drive folder for local storage, call it anything you want, will be created at top level of your drive
OUTPUT_DIR = "chatagent-scratch" #@param {type:"string"}

save_to_gdrive = False
if save_to_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')
    OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR
else:
    OUTPUT_DIR = "~/resum/" + OUTPUT_DIR

#@markdown Your OpenAI api key (can be obtained from https://platform.openai.com/account/api-keys, this notebook will be cheap, but not free)
OPENAI_API_KEY = "" #@param {type:"string"}
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

#@markdown Your SerpAPI api key (can be obtained from https://platform.openai.com/account/api-keys, this notebook will be cheap, but not free)
SERPAPI_API_KEY = "" #@param {type:"string"}
os.environ['SERPAPI_API_KEY'] = SERPAPI_API_KEY

#@markdown Your Wolfram-Alpha api key (can be obtained from https://platform.openai.com/account/api-keys, this notebook will be cheap, but not free)
WOLFRAM_ALPHA_APPID = "" #@param {type:"string"}
os.environ['WOLFRAM_ALPHA_APPID'] = WOLFRAM_ALPHA_APPID

!mkdir -p $OUTPUT_DIR

%cd $OUTPUT_DIR

print("Done!")

/Users/jim/resum/chatagent-scratch
Done!


In [2]:
# show google drive integration in sidebar

%cd $OUTPUT_DIR

import os.path

if not os.path.isfile(OUTPUT_DIR+"/app.py"):
  !git lfs install
  !git clone https://huggingface.co/spaces/JavaFXpert/Chat-GPT-LangChain $OUTPUT_DIR
else:
  print("Working directory looks good.")

try:
  from langchain.llms import OpenAI
  %pip install wolframalpha
  print("Dependencies look good.")
except:
  %cd $OUTPUT_DIR
  !git pull
  %pip install -r requirements.txt
  %pip install wolframalpha
  print("Dependencies look good.")

print("Done!")

/Users/jim/resum/chatagent-scratch
git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log
fatal: destination path '/Users/jim/resum/chatagent-scratch' already exists and is not an empty directory.
You should consider upgrading via the '/Users/jim/.pyenv/versions/3.10.1/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Dependencies look good.
Done!


In [3]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser, load_tools, initialize_agent
from langchain.prompts import BaseChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
import re

In [4]:
# Define which tools the agent can use to answer user queries

search = SerpAPIWrapper()
wolfram = WolframAlphaAPIWrapper()

tools = [
    Tool(
        name = "Search with SerpAPI",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name = "Ask WolframAPI",
        func=wolfram.run,
        description="useful for when you need to answer a question about math, numbers, logic or quantities"
    )
]

In [5]:
#@title Parse Agent Output
#@markdown This is just some regex stuff to see if 'Final Answer' appears and to extract what comes after it, nothing mysterious (at least no more mysterious than regex, haha)

class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [6]:
#@title Format Each Step in the Lang Chain
#@markdown This is fun. Here we can see what args the agent is using to make decisions at each step. All we're doing here is formatting them to make them nice to print at each step

# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

In [7]:
#@title Customize the base LangChain template and add pirate talk!

# THIS IS WHERE THE MAGIC HAPPENS
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
{agent_scratchpad}"""

prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

# Use the substitutions below to make the agent talk like a pirate
# ...best you can, but speaking as a pirate might speak
# ...Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

In [8]:
output_parser = CustomOutputParser()

# LLM chain consisting of the LLM and a prompt
llm = ChatOpenAI(temperature=0) # can easily set local model here as well
llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [9]:
# agent_executor.run("How many people live in canada as of 2023?")
# agent_executor.run("What is 37 x 455938")

# agent_executor.run("Write me a poem about cats")

# SPEAKER'S NOTES
# don't forget to make it talk like a pirate...
# explain llm-math SHOW CUSTOM PROMPT (non-deterministic programming): # https://python.langchain.com/en/latest/modules/chains/examples/llm_math.html

agent_executor.run("What is the eighth word in this sentence?")



> Entering new AgentExecutor chain...
Thought: This question requires simple counting and doesn't require any API. 
Action: Count the words in the sentence and find the eighth word.
Action Input: Count the words in the sentence.

Observation:Count the words in the sentence and find the eighth word. is not a valid tool, try another one.Oops, I don't need any API for this question. Let me just count the eighth word manually.
Action: Count the eighth word in the sentence.
Action Input: Manually count the eighth word.

Observation:Count the eighth word in the sentence. is not a valid tool, try another one.I don't need any API for this question, let me just count the eighth word manually.
Action: Count the eighth word in the sentence.
Action Input: Manually count the eighth word.

Observation:Count the eighth word in the sentence. is not a valid tool, try another one.Oops, my previous actions were not necessary. Let me just count the eighth word in the sentence.
Action: Count the eighth w

ValueError: Could not parse LLM output: `The final answer is "in".`

===================================================================
===================================================================


# TODOs

Implement embeddings and vector search
```
def FaissWrapper():
    foo = "bar"
    topic_search = FaissWrapper()
    tools = [
      Tool(
        name = "Search Sources",
        func=topic_search.run,
        description="useful for when you need to search embeddings"
    )
]
```

